In [ ]:
import boto3

In [ ]:
client = boto3.client('dynamodb',region_name='ap-east-1')

In [ ]:
table_name = 'Movies'

In [ ]:
attributes = [
    {
        'AttributeName':'Title',
        'AttributeType': 'S'
    },
    {
        'AttributeName':'Rating',
        'AttributeType': 'N'
    }
]

In [ ]:
key_schema = [
    {
        'AttributeName':'Title',
        'KeyType':'HASH'
    },
    {
        'AttributeName': 'Rating',
        'KeyType':'RANGE'
    }
]

In [ ]:
provisioned_throughput = {
    'ReadCapacityUnits':5,
    'WriteCapacityUnits':5
}

In [ ]:
response = client.create_table(TableName=table_name,
                              AttributeDefinitions=attributes,
                              KeySchema=key_schema,
                              ProvisionedThroughput=provisioned_throughput
                              )

In [ ]:
response

In [ ]:
entry = {'Title':{'S':'The Matrix'},
        'Director':{'S':'Lana Wachowski'},
        'Year':{'N':'1999'},
        'Rating':{'N':'5'}
        }

In [ ]:
client.put_item(TableName='Movies',Item=entry)

In [ ]:
item_key = {'Title':{'S':'The Matrix'},
        'Rating':{'N':'5'}
        }

In [ ]:
response = client.get_item(TableName='Movies',Key=item_key)

In [ ]:
response['Item']

In [ ]:
update = 'SET Director = :r'

In [ ]:
client.update_item(TableName='Movies',
                  Key=item_key,
                  UpdateExpression=update,
                  ExpressionAttributeValues={':r':{'S':'Lana and Lilly Wachowski'}}
                  )

In [ ]:
client.delete_item(TableName='Movies',
                  Key=item_key)

In [ ]:
movies = [
        {"Title": "The Matrix",
         "Director": "Lana Wachowski",
         "Year": "1999",
         "Rating": "4.7"},
    
        {"Title": "The Matrix 2",
             "Director": "Lana Wachowski",
             "Year": "2003",
             "Rating": "4.6"},

        {"Title": "The Matrix 3",
             "Director": "Lana Wachowski",
             "Year": "2003",
             "Rating": "4.5"},

        {"Title": "Inception",
             "Director": "Christopher Nolan",
             "Year": "2010",
             "Rating": "4.6"},
    
        {"Title": "Saving Private Ryan",
             "Director": "Steven Spielberg",
             "Year": "1999",
             "Rating": "4.7"},

]

In [ ]:
batch_request = []

for movie in movies:
    batch_request.append({
        'PutRequest':{
            'Item':{
                'Title':{'S':movie['Title']},
                'Rating':{'N':str(movie['Rating'])},
                'Director':{'S':movie['Director']},
                'Year':{'N':str(movie['Year'])}
            }
        }
    })

In [ ]:
batch_request

In [ ]:
response = client.batch_write_item(RequestItems={'Movies':batch_request})

In [ ]:
response

In [ ]:
batch_request_2 = {"Keys": []}
for movie in movies:
    batch_request_2["Keys"].append({
            'Title': {'S': movie['Title']},
            'Rating': {'N': str(movie['Rating'])},
        }
    )

In [ ]:
batch_request_2

In [ ]:
client.batch_get_item(RequestItems={'Movies':batch_request_2})

In [ ]:
items = []
response = client.scan(TableName='Movies')
items.extend(response['Items'])

while "LastEvaluatedKey" in response.keys():
    response = client.scan(TableName='Movies',ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response['Items'])

In [ ]:
items

In [ ]:
client.scan(TableName='Movies',
           FilterExpression='Rating >= :num',
           ExpressionAttributeValues={':num':{'N':'4.7'}})

In [ ]:
dynamodb = boto3.resource('dynamodb',region_name='ap-east-1')
table = dynamodb.Table('Movies')

In [ ]:
from boto3.dynamodb.conditions import Key, Attr

In [ ]:
dir(Attr)

In [ ]:
from decimal import Decimal
table.scan(FilterExpression=Attr("Rating").gte(Decimal(4.5)))

In [ ]:
table.key_schema

In [ ]:
client.describe_table(TableName='Movies')

In [ ]:
attributes

In [ ]:
attributes.append({"AttributeName":"Director",'AttributeType':'S'})

In [ ]:
attributes

In [ ]:
gsi_key_schema = [
    {
        'AttributeName':'Director',
        'KeyType':'HASH'
    }
]

In [ ]:
gsi_provisioned_throughput = {
    'ReadCapacityUnits':5,
    'WriteCapacityUnits':5
}

In [ ]:
response = client.update_table(TableName='Movies',
                              AttributeDefinitions=attributes,
                              GlobalSecondaryIndexUpdates=[
                                  {'Create':{
                                      'IndexName':'idx1',
                                      'KeySchema':gsi_key_schema,
                                      'Projection':{
                                          'ProjectionType':'ALL'
                                      },
                                      'ProvisionedThroughput':gsi_provisioned_throughput
                                  }
                                  }
                              ])

In [ ]:
client.query(TableName='Movies',
            KeyConditionExpression='Director = :d',
            IndexName='idx1',
            ExpressionAttributeValues={':d':{'S':'Steven Spielberg'}})